In [137]:
from langchain.vectorstores import FAISS
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import Graph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Any
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from semantic_text_splitter import TextSplitter
from dotenv import load_dotenv
import os
import re
import tiktoken
import pdfplumber
import json

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
os.environ['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY")

# Set the model as ChatOpenAI
model = ChatOpenAI(model_name="gpt-4", temperature=0)

pdfs_path = 'pdfs/beroepsproducten/temp/'

# List all files in the given folder
files = os.listdir(pdfs_path)

# Filter out all files that are not PDFs
pdf_docs = [f for f in files if f.endswith('.pdf')]


class Topic(BaseModel):
    topic: str = Field(description="Onderwerp van de tekst") 
    summary: str = Field(description="Samenvatting van de tekst")
    feedback: str = Field(description="Feedback")
    vragen: str = Field(description="Verdiepende vragen")

In [138]:
def get_pdf_text(pdfs_path, pdf_docs):
    text = ""
    for pdf_doc in pdf_docs:
        pdf_path = os.path.join(pdfs_path, pdf_doc)
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                # Extract text
                page_text = page.extract_text(layout=True)
                if page_text:
                    text += page_text + "\n"

                # Extract tables and handle None values
                table = page.extract_table()
                if table:
                    for row in table:
                        # Convert each cell to a string, replacing None with an empty string
                        cleaned_row = [cell if cell is not None else "" for cell in row]
                        text += ' | '.join(cleaned_row) + "\n"
    return text

# def get_text_chunks(text):
#     # Maximum number of tokens in a chunk
#     max_tokens = 500
#     text_splitter = TextSplitter.from_tiktoken_model("gpt-3.5-turbo")
#     chunks = text_splitter.chunks(text, max_tokens)
#     return chunks

def get_text_chunks(text):
    text_splitter = TextSplitter.from_tiktoken_model("gpt-3.5-turbo")
    chunks = text_splitter.chunks(text, chunk_capacity=(500,1000))
    return chunks

# get pdf text
raw_text = get_pdf_text(pdfs_path, pdf_docs)
cleaned_text = re.sub(r'\s+', ' ', raw_text.strip())

# get the text chunks
text_chunks = get_text_chunks(cleaned_text)

for chunk in text_chunks: print(chunk + "\n")

ABSTRACT Plaatje is AI gegenereerd door Dall- E Maurice van Ha2em 12-03-2024 MASTER MODULE DIGITALE INN O VATIE Inhoud Beroepsopdracht 3 ..................................................................................................................................2 Teamvorming en Kick-off .....................................................................................................................2 Doelstelling: .........................................................................................................................................2 Hoe organiseren we co-creaQe? ..........................................................................................................3 Hoe richten we het ontwerpproces in? ...............................................................................................3 Hoe meten we voortgang en succes? ..................................................................................................3 Welke faciliteiten en resourc

In [145]:
def agent_assessor(docs):
    # print(subtopics)
    # Set up a parser + inject instructions into the prompt template.
    parser = JsonOutputParser(pydantic_object=Topic)
    review= []
    for doc in docs:
        agent_reader_query = "Jouw taak is om het belangrijkste onderwerp te benoemen in deze tekst: " + doc + ".\
            Daarnaast moet je een samenvatting geven van de tekst en feedback geven op de tekst. Als laatste stel je één \
            of meer verdiepende vragen waarmee het onderwerp verder onderzocht kan worden. Genereer alleen NEDERLANDSE teksten.\
            Als er in de tekst bedrijfsnamen of persoonsnamen voorkomen, vervang deze dan door 'bedrijf X' of 'persoon N'."
        prompt = PromptTemplate(
            template="Voer de opdracht uit.\n{format_instructions}\n{query}\n",
            input_variables=["query"],
            partial_variables={"format_instructions": parser.get_format_instructions()},
        )
        chain = prompt | model | parser
        result = chain.invoke({"query": agent_reader_query})
        review.append(result)
        print(result)
    with open('topics.json', 'w') as f:
        json.dump(review, f, indent=4)

In [146]:
agent_assessor(text_chunks)

{'topic': "Digitale innovatie en implementatie van intelligente camera's in bedrijf XXX", 'summary': "De tekst gaat over de uitdagingen op de arbeidsmarkt en de mogelijkheden die nieuwe technologieën bieden om processen in bedrijf XXX te optimaliseren. De auteur stelt voor om intelligente camera's te implementeren voor voorraadbeheer en diefstalpreventie. Om deze digitale innovatie te realiseren, is een multidisciplinair team nodig. De auteur kiest voor de 'Design Thinking' ontwerpmethode om deze innovatie verder uit te werken.", 'feedback': "De tekst is goed gestructureerd en het doel is duidelijk. Het zou echter nuttig zijn om meer specifieke details te geven over hoe de implementatie van intelligente camera's zou werken en welke stappen er nodig zijn om dit te bereiken. Bovendien zou het nuttig zijn om meer te weten te komen over de rol van elk teamlid in dit proces.", 'vragen': "Hoe ziet het implementatieproces van de intelligente camera's er precies uit? Welke uitdagingen kunnen e